In [3]:
import tensorflow as tf
import numpy as np
import os

from data_frame import DataFrame
from tf_utils import lstm_layer, time_distributed_dense_layer, sequence_log_loss
from tf_base_model import TFBaseModel

In [5]:
class DataReader(object):

    def __init__(self, data_dir):
#         data_cols = [
#             'ip_id',
#             'app_id',
#             'channel_id',
#             'os_id',
#             'device_id',
#             'ip_attributed_history',
#             'app_attributed_history',
#             'channel_attributed_history',
#             'os_attributed_history',
#             'device_attributed_history',
#             'ip_time_delta_history',
#             'app_time_delta_history',
#             'channel_time_delta_history',
#             'os_time_delta_history',
#             'device_time_delta_history',
#             'ip_history_length',
#             'app_history_length',
#             'channel_history_length',
#             'os_history_length',
#             'device_history_length',
#         ]
        data_cols = [
                'ip_id',
                'ip_attributed_history',
                'ip_time_delta_history',
                'ip_history_length',
                'ip_next_download'
            ]
        data = [np.load(os.path.join(data_dir, '{}.npy'.format(i)), mmap_mode='r') for i in data_cols]
        self.test_df = DataFrame(columns=data_cols, data=data)

        print(self.test_df.shapes())
        print('loaded data')

        self.train_df, self.val_df = self.test_df.train_test_split(train_size=0.9)

        print('train size', len(self.train_df))
        print('val size', len(self.val_df))
        print('test size', len(self.test_df))

    def train_batch_generator(self, batch_size):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.train_df,
            shuffle=True,
            num_epochs=10000,
            is_test=False
        )

    def val_batch_generator(self, batch_size):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.val_df,
            shuffle=True,
            num_epochs=10000,
            is_test=False
        )

    def test_batch_generator(self, batch_size):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.test_df,
            shuffle=False,
            num_epochs=1,
            is_test=True
        )

    def batch_generator(self, batch_size, df, shuffle=True, num_epochs=10000, is_test=False):
        batch_gen = df.batch_generator(batch_size, shuffle=shuffle, num_epochs=num_epochs, allow_smaller_final_batch=is_test)
        for batch in batch_gen:
            batch['ip_attributed_history'] = np.roll(batch['ip_attributed_history'], -1, axis=1)
            batch['ip_time_delta_history'] = np.roll(batch['ip_time_delta_history'], -1, axis=1)
            batch['ip_next_download'] = batch['ip_next_download'] - 1
            if not is_test:
                batch['ip_history_length'] = batch['ip_history_length'] - 1
            yield batch


class rnn(TFBaseModel):

    def __init__(self, lstm_size=300, **kwargs):
        self.lstm_size = lstm_size
        super(rnn, self).__init__(**kwargs)

    def calculate_loss(self):
        x = self.get_input_sequences()
        preds = self.calculate_outputs(x)
        loss = sequence_log_loss(self.ip_next_download, preds, self.ip_history_length, 1000)
        return loss

    def get_input_sequences(self):
        self.ip_id = tf.placeholder(tf.int32, [None])
        self.ip_history_length = tf.placeholder(tf.int32, [None])
        self.ip_next_download = tf.placeholder(tf.int32, [None])

        self.ip_attributed_history = tf.placeholder(tf.int32, [None, 1000])
        self.ip_time_delta_history = tf.placeholder(tf.int32, [None, 1000])

        self.keep_prob = tf.placeholder(tf.float32)
        self.is_training = tf.placeholder(tf.bool)

        # ip data
        ip_embeddings = tf.get_variable(
            name='ip_embeddings',
            shape=[277397, self.lstm_size],
            dtype=tf.float32
        )
        x_ip = tf.nn.embedding_lookup(ip_embeddings, self.ip_id)
        x_ip = tf.tile(tf.expand_dims(x_ip, 1), (1, 1000, 1))

        # sequence data
        ip_attributed_history = tf.one_hot(self.ip_attributed_history, 2)
        ip_time_delta_history = tf.one_hot(self.ip_time_delta_history, 300)

        x_history = tf.concat([
            ip_attributed_history,
            ip_time_delta_history,
        ], axis=2)

        x = tf.concat([x_history, x_ip], axis=2)

        return x

    def calculate_outputs(self, x):
        h = lstm_layer(x, self.ip_history_length, self.lstm_size, scope='lstm1')
        h = tf.concat([h, x], axis=2)

        self.h_final = time_distributed_dense_layer(h, 50, activation=tf.nn.relu, scope='dense1')
        y_hat = tf.squeeze(time_distributed_dense_layer(self.h_final, 1, activation=tf.nn.sigmoid, scope='dense2'), 2)

        final_temporal_idx = tf.stack([tf.range(tf.shape(self.ip_history_length)[0]), self.ip_history_length - 1], axis=1)
        self.final_states = tf.gather_nd(self.h_final, final_temporal_idx)
        self.final_predictions = tf.gather_nd(y_hat, final_temporal_idx)

        self.prediction_tensors = {
            'ip_ids': self.ip_id,
            'final_states': self.final_states,
            'predictions': self.final_predictions
        }

        return y_hat


if __name__ == '__main__':
    base_dir = './'

    dr = DataReader(data_dir=os.path.join(base_dir, '..','input', 'processed'))

    nn = rnn(
        reader=dr,
        log_dir=os.path.join(base_dir, 'logs'),
        checkpoint_dir=os.path.join(base_dir, 'checkpoints'),
        prediction_dir=os.path.join(base_dir, 'predictions'),
        optimizer='adam',
        learning_rate=.001,
        lstm_size=300,
        batch_size=128,
        num_training_steps=200000,
        early_stopping_steps=20000,
        warm_start_init_step=0,
        regularization_constant=0.0,
        keep_prob=1.0,
        enable_parameter_averaging=False,
        num_restarts=0,
        min_steps_to_checkpoint=2500,
        log_interval=20,
        num_validation_batches=4,
    )
    nn.fit()
    nn.restore()
    nn.predict()

ip_attributed_history    (277396, 1000)
ip_history_length             (277396,)
ip_id                         (277396,)
ip_next_download              (277396,)
ip_time_delta_history    (277396, 1000)
dtype: object
loaded data


/home/mohsin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)

new run with parameters:
{'batch_size': 128,
 'checkpoint_dir': './checkpoints',
 'early_stopping_steps': 20000,
 'enable_parameter_averaging': False,
 'grad_clip': 5,
 'keep_prob_scalar': 1.0,
 'learning_rate': 0.001,
 'log_dir': './logs',
 'log_interval': 20,
 'loss_averaging_window': 100,
 'lstm_size': 300,
 'min_steps_to_checkpoint': 2500,
 'num_restarts': 0,
 'num_training_steps': 200000,
 'num_validation_batches': 4,
 'optimizer': 'adam',
 'prediction_dir': './predictions',
 'reader': <__main__.DataReader object at 0x7f40481f6898>,
 'regularization_constant': 0.0,
 'warm_start_init_step': 0}

new run with parameters:
{'batch_size': 128,
 'checkpoint_dir': './checkpoints',
 'early_stopping_steps': 20000,
 'enable_parameter_averaging': False,
 'grad_clip': 5,
 'keep_prob_

train size 249656
val size 27740
test size 277396


all parameters:
all parameters:
all parameters:
[('ip_embeddings:0', [277397, 300]),
 ('lstm1/rnn/lstm_cell/kernel:0', [902, 1200]),
 ('lstm1/rnn/lstm_cell/bias:0', [1200]),
 ('dense1/weights:0', [902, 50]),
 ('dense1/biases:0', [50]),
 ('dense2/weights:0', [50, 1]),
 ('dense2/biases:0', [1]),
 ('Variable:0', []),
 ('Variable_1:0', []),
 ('beta1_power:0', []),
 ('beta2_power:0', []),
 ('ip_embeddings/Adam:0', [277397, 300]),
 ('ip_embeddings/Adam_1:0', [277397, 300]),
 ('lstm1/rnn/lstm_cell/kernel/Adam:0', [902, 1200]),
 ('lstm1/rnn/lstm_cell/kernel/Adam_1:0', [902, 1200]),
 ('lstm1/rnn/lstm_cell/bias/Adam:0', [1200]),
 ('lstm1/rnn/lstm_cell/bias/Adam_1:0', [1200]),
 ('dense1/weights/Adam:0', [902, 50]),
 ('dense1/weights/Adam_1:0', [902, 50]),
 ('dense1/biases/Adam:0', [50]),
 ('dense1/biases/Adam_1:0', [50]),
 ('dense2/weights/Adam:0', [50, 1]),
 ('dense2/weights/Adam_1:0', [50, 1]),
 ('dense2/biases/Adam:0', [1]),
 ('dense2/biases/Adam_1:0', [1])]
[('ip_embeddings:0', [277397, 300])

built graph


InvalidArgumentError: Incompatible shapes: [512] vs. [512,1000]
	 [[Node: mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Cast, Log)]]
	 [[Node: truediv/_37 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_294_truediv", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'mul', defined at:
  File "/home/mohsin/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mohsin/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mohsin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mohsin/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.s